### JsonOutputParser

사용자가 원하는 JSON 스키마를 지정할 수 있게 해주는 도구. LLM이 데이터를 조회하고 결과를 도출할 때, 지정된 스키마에 맞게 JSON 형식으로 데이터를 반환할 수 있도록 설계

LLM이 데이터를 정확하고 효율적으로 처리하여 사용자가 원하는 형태의 JSON을 생성하기 위해서는 모델의 parameter가 충분히 커야 함.

In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

logging.langsmith("JsonOuputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
JsonOuputParser


In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [3]:
model = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature=0
    )

In [5]:
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [10]:
output_parser = JsonOutputParser(pydantic_object=Topic)

question = "지구 온난화의 심각성에 대해 알려주세요."

prompt = ChatPromptTemplate(
    [
        ("system", "당신은 친절한 AI 어시스턴스입니다. 질문에 간결하게 답변하세요."),
        ("user","#Format: {format_instcutions}\n\n#Question: {question}"),
    ]
)
prompt = prompt.partial(format_instcutions= output_parser.get_format_instructions())
chain = prompt | model | output_parser

answer = chain.invoke({"question": question})

In [11]:
print(answer)
print(type(answer))

{'description': '지구 온난화는 기후 변화의 주요 원인으로, 평균 기온 상승과 극단적인 기상 현상을 초래하고 있습니다.', 'hashtags': '#지구온난화 #기후변화'}
<class 'dict'>


###  Pydnatic 없이 사용.

In [12]:
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 'description'에, 관련 키워드는 'hashtags'에 담아주세요."

parser = JsonOutputParser()

prompt = ChatPromptTemplate(
    [
        ("system", "당신은 친절한 AI 어시스턴스입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instcutions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instcutions= parser.get_format_instructions())

chain = prompt | model | parser
chain.invoke({"question": question})

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 온실가스의 증가로 인해 발생합니다. 이로 인해 기후 변화, 해수면 상승, 생태계 변화 등의 문제가 발생하고 있습니다.',
 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#지구환경']}